In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd
from IPython.display import IFrame
%matplotlib inline
ox.__version__

'1.0.0'

In [2]:
# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

In [3]:
#read the files from the Colonias folder using geopandas just as if we were using only pandas
colonias = gpd.read_file('input_data/coloniasmexico/Colonias/')

In [4]:
#query to filter only the colonias from Merida municipality of Yucatan state
colonias_yucatan_merida = colonias[(colonias['ST_NAME']=='YUCATAN') & (colonias['MUN_NAME'] == 'MÉRIDA')]

In [5]:
colonias_yucatan_merida.head(3)

,OBJECTID,POSTALCODE,ST_NAME,MUN_NAME,SETT_NAME,SETT_TYPE,AREA,Shape_Leng,Shape_Area,geometry
56515,56516,97000,YUCATAN,MÉRIDA,BARRIO SAN SEBASTIAN,FRACCIONAMIENTO,724836804.0,0.028660,0.000038,"POLYGON Z ((-89.63250 20.96072 0.00000, -89.63..."
56516,56517,97000,YUCATAN,MÉRIDA,BARRIO SANTA ANA,FRACCIONAMIENTO,724836804.0,0.031112,0.000051,"POLYGON Z ((-89.62424 20.97568 0.00000, -89.62..."
56517,56518,97000,YUCATAN,MÉRIDA,BARRIO SANTIAGO,FRACCIONAMIENTO,724836804.0,0.023338,0.000034,"POLYGON Z ((-89.62914 20.96978 0.00000, -89.62..."


In [6]:
# save/load graph as a graphml file: this is the best way to save your model
# for subsequent work later
# if you want to work with your model in gephi, use gephi compatibility mode

filepath = './networks/merida-kanasin-road.graphml'
#ox.save_graphml(G, filepath=filepath, gephi=False)
G = ox.load_graphml(filepath)

In [7]:
#extract nodes as a geodataframe
nodes = ox.graph_to_gdfs(G, edges=False)

In [48]:
# for each colonia polygon, find intersecting nodes then induce subgraph
j = 0
#subgraphs_colonias = [] #vector for storing subgraphs
stats = []
vec = []
for i in range(colonias_yucatan_merida.size):
    j = j+1
    if j == 10: #only the first 10 colonia; just for testing
        break
        
    polygon = colonias_yucatan_merida.iloc[i].geometry
    colonia_name = colonias_yucatan_merida.iloc[i].SETT_NAME
    mun_name = colonias_yucatan_merida.iloc[i].MUN_NAME
    ob_id = colonias_yucatan_merida.iloc[i].OBJECTID

    
    try: 
        #try/except for handling empty graphs; 
        #if the graph is empty, NetworkX raises an error and stops the loop;
        #this code ignores the empty graphs in the except statement
        intersecting_nodes = nodes[nodes.intersects(polygon)].index
        G_sub = G.subgraph(intersecting_nodes)
        #subgraphs_colonias.append(G_sub)
        #print(colonia_name+", "+mun_name)
        stat = ox.basic_stats(G_sub)
        extended_stat = ox.extended_stats(G_sub, ecc=True, bc=True, cc=True)
        for key, value in extended_stat.items():
            stat[key] = value
        vec = [stat[elem] for elem in stat if isinstance(stat[elem], dict) == False]
        stats.append(vec)
        #fig, ax = ox.plot_graph(G_sub)
    except:
        pass

In [49]:
stats

[[30,
  38,
  2.533333333333333,
  28,
  3.466666666666667,
  4427.6089999999995,
  116.51602631578946,
  4288.085,
  119.11347222222223,
  36,
  None,
  None,
  None,
  None,
  1.0001022692745687,
  0.0,
  None,
  None,
  1.2,
  0.018237888316205802,
  0.08735632183908044,
  0.0,
  0.0,
  287661241,
  0.08217985906060121,
  315231835,
  0.006920679467969117,
  1286.675,
  727.998,
  [287661241],
  [6363345517],
  0.000952143919323382,
  0.07032019704433498],
 [46,
  64,
  2.782608695652174,
  38,
  3.1956521739130435,
  6807.298999999999,
  106.36404687499999,
  6392.377999999999,
  112.14698245614034,
  57,
  None,
  None,
  None,
  None,
  1.0006172035802712,
  0.0,
  None,
  None,
  1.3695652173913044,
  0.017569600457330473,
  0.06183574879227054,
  0.0,
  0.0,
  6365185635,
  0.0685027196944954,
  8238384011,
  0.004334442062122589,
  1400.7910000000002,
  876.9119999999999,
  [831384211],
  [6365185631],
  0.000996051130021163,
  0.06639218269653054],
 [30,
  43,
  2.86666666666

In [50]:
len(stats)

7

In [9]:
len(subgraphs_colonias)

9

In [ ]:
#circ = []
#for j in (len(city)):
#    intersecting_nodes = nodes[nodes.intersects(j)].index
#    neighbourhood_graph = city_graph.subgraph(intersecting_nodes)
#    stat = ox.basic_stats(neighbourhood_graph)
#    circ.append(stat['circuity_avg'])

In [76]:
# calculate basic and extended network stats, merge them together, and display
stats = ox.basic_stats(subgraphs_colonias[0])
extended_stats = ox.extended_stats(subgraphs_colonias[0], ecc=True, bc=True, cc=True)
for key, value in extended_stats.items():
    stats[key] = value
pd.Series(stats)

n                                                                                     30
m                                                                                     38
k_avg                                                                           2.533333
intersection_count                                                                    28
streets_per_node_avg                                                            3.466667
streets_per_node_counts                                 {0: 0, 1: 2, 2: 0, 3: 10, 4: 18}
streets_per_node_proportion            {0: 0.0, 1: 0.06666666666666667, 2: 0.0, 3: 0....
edge_length_total                                                            4427.609000
edge_length_avg                                                               116.516026
street_length_total                                                          4288.085000
street_length_avg                                                             119.113472
street_segments_count

Streets/intersection counts and proportions are nested dicts inside the stats dict. To convert these stats to a pandas dataframe (to compare/analyze multiple networks against each other), just unpack these nested dicts first:

In [77]:
# unpack dicts into individiual keys:values
stats = ox.basic_stats(subgraphs_colonias[0])
for k, count in stats['streets_per_node_counts'].items():
    stats['int_{}_count'.format(k)] = count
for k, proportion in stats['streets_per_node_proportion'].items():
    stats['int_{}_prop'.format(k)] = proportion

# delete the no longer needed dict elements
del stats['streets_per_node_counts']
del stats['streets_per_node_proportion']

# load as a pandas dataframe
pd.DataFrame(pd.Series(stats, name='value'))

,value
n,30.000000
m,38.000000
k_avg,2.533333
intersection_count,28.000000
streets_per_node_avg,3.466667
edge_length_total,4427.609000
edge_length_avg,116.516026
street_length_total,4288.085000
street_length_avg,119.113472
street_segments_count,36.000000


In [61]:
vector = []
for subgraph in subgraphs_colonias:
    if subgraph.number_of_edges() != 0:
        stats = ox.basic_stats(subgraph)
        [vector.append(stats[elem]) for elem in stats]
        #[vector.append(colonia1[elem])]
    else:
        pass
print(vector)

[30, 38, 2.533333333333333, 28, 3.466666666666667, {0: 0, 1: 2, 2: 0, 3: 10, 4: 18}, {0: 0.0, 1: 0.06666666666666667, 2: 0.0, 3: 0.3333333333333333, 4: 0.6}, 4427.6089999999995, 116.51602631578946, 4288.085, 119.11347222222223, 36, None, None, None, None, 1.0001022692745687, 0.0, None, None, 46, 64, 2.782608695652174, 38, 3.1956521739130435, {0: 0, 1: 8, 2: 0, 3: 13, 4: 25}, {0: 0.0, 1: 0.17391304347826086, 2: 0.0, 3: 0.2826086956521739, 4: 0.5434782608695652}, 6807.298999999999, 106.36404687499999, 6392.377999999999, 112.14698245614034, 57, None, None, None, None, 1.0006172035802712, 0.0, None, None, 30, 43, 2.8666666666666667, 30, 3.8333333333333335, {0: 0, 1: 0, 2: 0, 3: 6, 4: 23, 5: 1}, {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.2, 4: 0.7666666666666667, 5: 0.03333333333333333}, 5078.082000000001, 118.09493023255817, 5078.081999999999, 118.09493023255813, 43, None, None, None, None, 1.0000474417908707, 0.0, None, None, 8, 11, 2.75, 4, 2.0, {0: 0, 1: 4, 2: 0, 3: 4}, {0: 0.0, 1: 0.5, 2: 0.0, 3: 0